In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from geopy.distance import vincenty

In [2]:
# load train and test data
df_mess_train = pd.read_csv('mess_train_list.csv')
df_mess_test = pd.read_csv('mess_test_list.csv')
pos_train = pd.read_csv('pos_train_list.csv')

In [3]:
df_mess_train.head()

,objid,bsid,did,nseq,rssi,time_ux,bs_lat,bs_lng
0,573bf1d9864fce1a9af8c5c9,2841,473335.0,0.5,-121.5,1.463546e+12,39.617794,-104.954917
1,573bf1d9864fce1a9af8c5c9,3526,473335.0,2.0,-125.0,1.463546e+12,39.677251,-104.952721
2,573bf3533e952e19126b256a,2605,473335.0,1.0,-134.0,1.463547e+12,39.612745,-105.008827
3,573c0cd0f0fe6e735a699b93,2610,473953.0,2.0,-132.0,1.463553e+12,39.797969,-105.073460
4,573c0cd0f0fe6e735a699b93,3574,473953.0,1.0,-120.0,1.463553e+12,39.723151,-104.956216


In [4]:
pos_train.head()

,lat,lng
0,39.606690,-104.958490
1,39.606690,-104.958490
2,39.637741,-104.958554
3,39.730417,-104.968940
4,39.730417,-104.968940


In [5]:
listOfBs = np.union1d(np.unique(df_mess_train['bsid']), np.unique(df_mess_test['bsid'])) # determine all Base stations that received at least 1 message

In [6]:
len(listOfBs)

259

In [131]:
np.max([10,100,1000])

1000

In [156]:
(10**(-121.5))/(10**(-121.5))

1.0

In [158]:
(10**(-125.))/(10**(-121.5))

0.00031622776601683794

In [159]:
# Feature Matrix construction 

def feat_mat_const(df_mess_train, listOfBs):

    df_mess_bs_group = df_mess_train.groupby(['objid'], as_index=False) # group data by message (objid)
    nb_mess = len(np.unique(df_mess_train['objid']))
    df_feat = pd.DataFrame(np.zeros((nb_mess,len(listOfBs))), columns = listOfBs)
    df_feat.loc[:,:] = np.nan# feature matrix
    idx = 0

    for key, elmt in df_mess_bs_group:
        df_mess_bs_group.get_group(key)
        vals = []
        if len(df_mess_bs_group.get_group(key)['rssi'].values) > 0:
            print(df_mess_bs_group.get_group(key)['rssi'].values)
            maximum = np.max(10**(df_mess_bs_group.get_group(key)['rssi'].values))
            print(maximum)
            vals = 10**df_mess_bs_group.get_group(key)['rssi'].values/maximum
            print(vals)
        break
        #df_feat.loc[idx,df_mess_bs_group.get_group(key)['bsid']] = 
        #idx = idx + 1
    #return df_feat

In [160]:
df_feat = feat_mat_const(df_mess_train, listOfBs)
#df_feat.head()

[-121.5 -125. ]
3.16227766017e-122
[  1.00000000e+00   3.16227766e-04]


In [119]:
def lat_mat_const(df_mess_train, listOfBs):

    df_mess_bs_group = df_mess_train.groupby(['objid'], as_index=False) # group data by message (objid)
    nb_mess = len(np.unique(df_mess_train['objid']))
    df_feat = pd.DataFrame(np.zeros((nb_mess,len(listOfBs))), columns = listOfBs) # feature matrix
    df_feat.loc[:,:] = np.nan
    idx = 0

    for key, elmt in df_mess_bs_group:
        df_mess_bs_group.get_group(key)
        df_feat.loc[idx,df_mess_bs_group.get_group(key)['bsid']] = df_mess_bs_group.get_group(key)['bs_lat'].values
        idx = idx + 1
    return df_feat

In [120]:
df_lat = lat_mat_const(df_mess_train, listOfBs)
df_lat.head()

,879,911,921,944,980,1012,1086,1092,1120,1131,...,9936,9941,9949,10134,10148,10151,10162,10999,11007,11951
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,39.973995,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
def lng_mat_const(df_mess_train, listOfBs):

    df_mess_bs_group = df_mess_train.groupby(['objid'], as_index=False) # group data by message (objid)
    nb_mess = len(np.unique(df_mess_train['objid']))
    df_feat = pd.DataFrame(np.zeros((nb_mess,len(listOfBs))), columns = listOfBs) # feature matrix
    df_feat.loc[:,:] = np.nan
    idx = 0

    for key, elmt in df_mess_bs_group:
        df_mess_bs_group.get_group(key)
        df_feat.loc[idx,df_mess_bs_group.get_group(key)['bsid']] = df_mess_bs_group.get_group(key)['bs_lng'].values
        idx = idx + 1
    return df_feat

In [122]:
df_lng = lng_mat_const(df_mess_train, listOfBs)
df_lng.head()

,879,911,921,944,980,1012,1086,1092,1120,1131,...,9936,9941,9949,10134,10148,10151,10162,10999,11007,11951
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,-104.891717,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
# ground truth construction

def ground_truth_const(df_mess_train, pos_train):

    df_mess_pos = df_mess_train.copy()
    df_mess_pos[['lat', 'lng']] = pos_train

    ground_truth_lat = np.array(df_mess_pos.groupby(['objid']).mean()['lat'])
    ground_truth_lng = np.array(df_mess_pos.groupby(['objid']).mean()['lng'])
    
    return ground_truth_lat, ground_truth_lng

In [124]:
ground_truth_lat, ground_truth_lng = ground_truth_const(df_mess_train, pos_train)
ground_truth_lat.shape

(6068,)

In [126]:
df_feat.replace(np.nan, (-10**6),inplace=True)

In [127]:
df_feat.reset_index(inplace=True)
df_lat.reset_index(inplace=True)
df_lng.reset_index(inplace=True)

In [128]:
data_lat = pd.merge(df_feat,df_lat,how='outer',on='index')
data_lng = pd.merge(df_feat,df_lng,how='outer',on='index')

In [129]:
data_lat

,index,879_x,911_x,921_x,944_x,980_x,1012_x,1086_x,1092_x,1120_x,...,9936_y,9941_y,9949_y,10134_y,10148_y,10151_y,10162_y,10999_y,11007_y,11951_y
0,0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-141.0,-1000000.0,-1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
def vincenty_vec(vec_coord):
    vin_vec_dist = np.zeros(vec_coord.shape[0])
    if vec_coord.shape[1] !=  4:
        print('ERROR: Bad number of columns (shall be = 4)')
    else:
        vin_vec_dist = [vincenty(vec_coord[m,0:2],vec_coord[m,2:]).meters for m in range(vec_coord.shape[0])]
    return vin_vec_dist

In [95]:
# evaluate distance error for each predicted point
def Eval_geoloc(y_train_lat , y_train_lng, y_pred_lat, y_pred_lng):
    vec_coord = np.array([y_train_lat , y_train_lng, y_pred_lat, y_pred_lng])
    err_vec = vincenty_vec(np.transpose(vec_coord))
    
    return err_vec

In [ ]:
err_vec = Eval_geoloc(ground_truth_lat , ground_truth_lng, y_pred_lat, y_pred_lng)

# Plot error distribution

In [ ]:
values, base = np.histogram(err_vec, bins=50000)
cumulative = np.cumsum(values) 
plt.figure();
plt.plot(base[:-1]/1000, cumulative / np.float(np.sum(values))  * 100.0, c='blue')
plt.grid(); plt.xlabel('Distance Error (km)'); plt.ylabel('Cum proba (%)'); plt.axis([0, 30, 0, 100]); 
plt.title('Error Cumulative Probability'); plt.legend( ["Opt LLR", "LLR 95", "LLR 99"])

## Error criterion

In [ ]:
np.percentile(err_vec, 80)

# Rastel

In [146]:
distances = list(map(vincenty, list(zip(df_mess_train.bs_lat, df_mess_train.bs_lng )),
list(zip(pos_train.lat, pos_train.lng ))))
distances = [x.miles for x in distances]
distances

[0.7894381445197945,
 4.877716425641559,
 3.1886145793460883,
 7.258708352654285,
 0.8431150795801082,
 0.2749054077285441,
 0.954905895099535,
 5.544682899261077,
 3.294204373577931,
 13.87143875889649,
 17.130405287759633,
 3.434038357741281,
 3.341841572011803,
 3.4487987227742165,
 11.739146529693926,
 4.0167917166589735,
 4.104090304887826,
 10.281629970704866,
 2.9946393833943685,
 1.6193655378421852,
 1.6974475093600854,
 2.1196071568009014,
 5.867810539166384,
 6.501032769513448,
 10.134435657618422,
 2.916669898638477,
 5.346456040299531,
 7.654440707369925,
 0.3450682887919251,
 4.625553295492139,
 4.5114560356289495,
 2.3672735738026716,
 8.299192153125848,
 1.5581502335297925,
 4.181597291080356,
 2.771627091633934,
 10.167473080751611,
 0.46402700071765496,
 2.887144286672691,
 2.8043700826541866,
 2.4094488643616936,
 1.395589529119549,
 5.215013714860038,
 7.173027203972747,
 3.3038440357152785,
 1.3662703698190926,
 3.988501671346246,
 2.6424877655203494,
 5.34727121824

In [147]:
mean_dist = pd.DataFrame({"bsid":df_mess_train.bsid.values, "dist":distances}).groupby(["bsid"]).mean()

In [153]:
mean_dist

,dist
bsid,
879,11.742900
911,10.461494
921,22.260418
944,26.844054
980,14.643733
1012,4.741836
1086,16.206069
1092,2246.466046
1120,25.506730


In [149]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(2).fit(mean_dist)

In [150]:
cluster = gmm.predict(mean_dist)

In [151]:
cluster

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1])

In [152]:
cluster.sum()

27